In [ ]:
%pylab inline
import tensorflow as tf
import tensorflow_probability as tfp

## Tensorflow.random
[https://www.tensorflow.org/guide/random_numbers]
> TensorFlow provides a set of pseudo-random number generators (RNG), in the tf.random module.
> TensorFlow provides two approaches for controlling the random number generation process:
> Through the explicit use of tf.random.Generator objects. Each such object maintains a state (in tf.Variable) that will be changed after each number generation.
Through the purely-functional stateless random functions like tf.random.stateless_uniform. Calling these functions with the same arguments (which include the seed) and on the same device will always produce the same results.

### old tf1 rng, now discouraged

In [ ]:
tf.random.uniform(shape=[10]).numpy()

In [ ]:
def test(generator, name, remark, eyeballtest):
    print (" " + name + "() " + remark)

    print (" checking period ...")
    start = generator()
    max_steps = 100000
    for i in range(max_steps):
        if generator() == start:
            print (" repeats after " + repr(i) + " steps")
            break
    if i+1 >= max_steps:
        print (" period larger than " + repr(max_steps))

    start_time = time.perf_counter()
    bins = 100000
    print (" binning " + repr(max_steps) + " tries in " + repr(bins) + " bins")
    bin_a = np.zeros(bins)
    for i in range(max_steps):
        b = int(generator() * bins)
        if b >= 0 and b < bins:
            bin_a[b] += 1
    expect = max_steps / float(bins)
    chisqr = np.sum( (bin_a - expect)**2 ) / expect
    cpu_time = time.perf_counter() - start_time
    print (" chi-squar/d.o.f. = " + repr(chisqr/(bins-1.0)))
    print (" CPU time = " + repr(cpu_time) + " seconds")

    for i in range( len(eyeballtest) ):
        eyeballtest[i] = [generator(), generator()]

In [ ]:
def tfuniform():                             # tensorflow uniform
    #global poor_seed
    #seed = poor_seed
    return  tf.random.uniform(shape=[]).numpy()

In [ ]:
tfuniform_pairs = np.zeros( (100000, 2) )
test(tfuniform, "tfuniform", "tfuniform", tfuniform_pairs)

In [ ]:
f1 = plt.figure(1, figsize=(13,13))
plt.plot(tfuniform_pairs[:,0], tfuniform_pairs[:,1],'b.')
plt.show()

### new tf2 rng

In [ ]:
rn1 = tf.random.get_global_generator()
print(rn1.uniform(shape=[2, 3]))

In [ ]:
def tf2uniform():                             # tensorflow uniform
    #global poor_seed
    #seed = poor_seed
    return  rn1.uniform(shape=[]).numpy()


In [ ]:
tf2uniform_pairs = np.zeros( (100000, 2) )
test(tf2uniform, "tfuniform", "tfuniform", tf2uniform_pairs)

In [ ]:
f2 = plt.figure(1, figsize=(13,13))
plt.plot(tf2uniform_pairs[:,0], tf2uniform_pairs[:,1],'r.')
plt.show()

## using a seed

In [ ]:
rn2 = tf.random.Generator.from_seed(1, alg='philox')

Different algorithms possible
> the RNG algorithm. Possible values are tf.random.Algorithm.PHILOX for the Philox algorithm and tf.random.Algorithm.THREEFRY for the ThreeFry algorithm (see paper 'Parallel Random Numbers: As Easy as 1, 2, 3' [https://www.thesalmons.org/john/random123/papers/random123sc11.pdf]). The string names "philox" and "threefry" can also be used. Note PHILOX guarantees the same numbers are produced (given the same random state) across all architectures (CPU, GPU, XLA etc).

In [ ]:
def tf2seed():                             # tensorflow uniform
    #global poor_seed
    #seed = poor_seed
    return  rn2.uniform(shape=[]).numpy()

In [ ]:
tf2seed_pairs = np.zeros( (100000, 2) )
test(tf2seed, "tfseed", "tfseed", tf2seed_pairs)

In [ ]:
f3 = plt.figure(1, figsize=(13,13))
plt.plot(tf2seed_pairs[:,0], tf2seed_pairs[:,1],'g.')
plt.show()

## based on a non-deterministic state
> Another way to create a generator is with Generator.from_non_deterministic_state. A generator created this way will start from a non-deterministic state, depending on e.g. time and OS.

In [ ]:
rn3 = tf.random.Generator.from_non_deterministic_state()

In [ ]:
def tf2nondet():                             # tensorflow uniform
    #global poor_seed
    #seed = poor_seed
    return  rn3.uniform(shape=[]).numpy()

In [ ]:
tf2nondet_pairs = np.zeros( (100000, 2) )
test(tf2nondet, "tfnondet", "tfnondet", tf2nondet_pairs)

In [ ]:
f4 = plt.figure(1, figsize=(13,13))
plt.plot(tf2nondet_pairs[:,0], tf2nondet_pairs[:,1],'g.')
plt.show()